# Laboratorio 2: Armado de un esquema de aprendizaje automático

En el laboratorio final se espera que puedan poner en práctica los conocimientos adquiridos en el curso, trabajando con un conjunto de datos de clasificación.

El objetivo es que se introduzcan en el desarrollo de un esquema para hacer tareas de aprendizaje automático: selección de un modelo, ajuste de hiperparámetros y evaluación.

El conjunto de datos a utilizar está en `./data/loan_data.csv`. Si abren el archivo verán que al principio (las líneas que empiezan con `#`) describen el conjunto de datos y sus atributos (incluyendo el atributo de etiqueta o clase).

Se espera que hagan uso de las herramientas vistas en el curso. Se espera que hagan uso especialmente de las herramientas brindadas por `scikit-learn`.

In [ ]:
import numpy as np
import pandas as pd

# TODO: Agregar las librerías que hagan falta
from sklearn.model_selection import train_test_split

# Metrics 
from sklearn.metrics         import accuracy_score
from sklearn.metrics         import f1_score
from sklearn.metrics         import precision_score
from sklearn.metrics         import recall_score
from sklearn.metrics         import confusion_matrix

# Models selections
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split

# Classifiers
from sklearn.linear_model    import SGDClassifier
from sklearn.tree            import DecisionTreeClassifier

## Carga de datos y división en entrenamiento y evaluación

La celda siguiente se encarga de la carga de datos (haciendo uso de pandas). Estos serán los que se trabajarán en el resto del laboratorio.

In [ ]:
dataset = pd.read_csv("https://raw.githubusercontent.com/DiploDatos/IntroduccionAprendizajeAutomatico/master/data/loan_data.csv", comment="#")

# División entre instancias y etiquetas
X, y = dataset.iloc[:, 1:], dataset.TARGET

# división entre entrenamiento y evaluación
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0, stratify=y)


Documentación:

- https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html

## Ejercicio 1: Descripción de los Datos y la Tarea

Responder las siguientes preguntas:

1. ¿De qué se trata el conjunto de datos?
2. ¿Cuál es la variable objetivo que hay que predecir? ¿Qué significado tiene?
3. ¿Qué información (atributos) hay disponible para hacer la predicción?
4. ¿Qué atributos imagina ud. que son los más determinantes para la predicción?

**No hace falta escribir código para responder estas preguntas.**

### Respuestas ejercicio 1:
1. El conjunto de datos contiene información acerca de 5960 préstamos con garantía hipotecaria recientes.
2. La variable objetivo (BAD) es una variable binaria que indica si un solicitante finalmente incurrió en incumplimiento o fue gravemente moroso.
3. Atributos disponibles para hacer la predicción:
  - LOAN: cantidad de préstamo requerido.
  - MORTDUE: monto adeudado de la hipoteca existente.
  - VALUE: valor de la propiedad actual.
  - YOJ: cantidad de años en el presente trabajo del solicitante.
  - DEROG: mayor número de reportes morosos.
  - DELINQ: número de líneas de crédito morosas.
  - CLAGE: antigüedad de la línea de crédito más antigua en meses.
  - NINQ: número de líneas de crédito recientes.
  - CLNO: número de líneas de crédito.
  - DEBTINC: relación deuda-ingresos.

## Ejercicio 2: Predicción con Modelos Lineales

En este ejercicio se entrenarán modelos lineales de clasificación para predecir la variable objetivo.

Para ello, deberán utilizar la clase SGDClassifier de scikit-learn.

Documentación:
- https://scikit-learn.org/stable/modules/sgd.html
- https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.SGDClassifier.html


### Ejercicio 2.1: SGDClassifier con hiperparámetros por defecto

Entrenar y evaluar el clasificador SGDClassifier usando los valores por omisión de scikit-learn para todos los parámetros. Únicamente **fijar la semilla aleatoria** para hacer repetible el experimento.

Evaluar sobre el conjunto de **entrenamiento** y sobre el conjunto de **evaluación**, reportando:
- Accuracy
- Precision
- Recall
- F1
- matriz de confusión

In [ ]:
def bad_prediction(model):
  trained_model = model.fit(X_train, y_train)
  y_pred_train = trained_model.predict(X_train)
  y_pred_test  = trained_model.predict(X_test)
  return y_pred_train, y_pred_test

In [ ]:
def print_metricas(prediction):
  print('accuracy train: ', round(accuracy_score(y_train, prediction[0]), 2))
  print('precision train: ', round(precision_score(y_train, prediction[0]), 2))
  print('recall train: ', round(recall_score(y_train, prediction[0]), 2))
  print('f1_score train: ', round(f1_score(y_train, prediction[0]), 2))
  print('matriz de confusión train:')
  print(confusion_matrix(y_train, prediction[0]))
  print()
  print('accuracy test: ', round(accuracy_score(y_test, prediction[1]), 2))
  print('precision test: ', round(precision_score(y_test, prediction[1]), 2))
  print('recall test: ', round(recall_score(y_test, prediction[1]), 2))
  print('f1_score test: ', round(f1_score(y_test, prediction[1]), 2))
  print('matriz de confusión test:')
  print(confusion_matrix(y_test, prediction[1]))
  print()

In [ ]:
model = SGDClassifier(random_state=60)
predicted = bad_prediction(model)
print_metricas(predicted)

accuracy train:  0.83
precision train:  0.0
recall train:  0.0
f1_score train:  0.0
matriz de confusión train:
[[1236    0]
 [ 247    0]]

accuracy test:  0.83
precision test:  0.0
recall test:  0.0
f1_score test:  0.0
matriz de confusión test:
[[309   0]
 [ 62   0]]



/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


### Ejercicio 2.2: Ajuste de Hiperparámetros

Seleccionar valores para los hiperparámetros principales del SGDClassifier. Como mínimo, probar diferentes funciones de loss, tasas de entrenamiento y tasas de regularización.

Para ello, usar grid-search y 5-fold cross-validation sobre el conjunto de entrenamiento para explorar muchas combinaciones posibles de valores.

Reportar accuracy promedio y varianza para todas las configuraciones.

Para la mejor configuración encontrada, evaluar sobre el conjunto de **entrenamiento** y sobre el conjunto de **evaluación**, reportando:
- Accuracy
- Precision
- Recall
- F1
- matriz de confusión

Documentación:
- https://scikit-learn.org/stable/modules/grid_search.html
- https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html

In [ ]:
# diccionario de parámetros a evaluar en el modelo SGDClassifier

grilla_param_SGDC = {'loss'          : ['hinge','squared_loss', 'perceptron'],
                     'penalty'       : ['l2','l1','elasticnet'],
                     'learning_rate' : ['optimal','adaptive'],
                     'eta0'          : [0.1],
                     'max_iter'      : [10000],
                     'alpha'         : (np.random.choice(100, 5, np.random.seed(60))/10),
                     'l1_ratio'      : (np.random.choice(10, 2, np.random.seed(60))/10)
                     }

In [ ]:
# selección del mejor modelo de SGDClassifier según los parámetros evaluados

model = SGDClassifier(random_state=60)

grilla_sgdc = GridSearchCV(model, grilla_param_SGDC, scoring='accuracy')

grilla_sgdc.fit(X_train, y_train)

mejor_sgdc = grilla_sgdc.best_estimator_
print(mejor_sgdc)

# entrenamiento y evaluación del mejor modelo de SGDClassifier según los parámetros evaluados

predicted = bad_prediction(mejor_sgdc)
print_metricas(predicted)

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_stochastic_gradient.py:557: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_stochastic_gradient.py:557: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_stochastic_gradient.py:557: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_stochastic_gradient.py:557: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_mo

SGDClassifier(alpha=0.4, average=False, class_weight=None, early_stopping=False,
              epsilon=0.1, eta0=0.1, fit_intercept=True, l1_ratio=0.8,
              learning_rate='adaptive', loss='perceptron', max_iter=10000,
              n_iter_no_change=5, n_jobs=None, penalty='elasticnet',
              power_t=0.5, random_state=60, shuffle=True, tol=0.001,
              validation_fraction=0.1, verbose=0, warm_start=False)
accuracy train:  0.78
precision train:  0.28
recall train:  0.21
f1_score train:  0.24
matriz de confusión train:
[[1104  132]
 [ 195   52]]

accuracy test:  0.77
precision test:  0.26
recall test:  0.19
f1_score test:  0.22
matriz de confusión test:
[[275  34]
 [ 50  12]]



In [ ]:
# accuracy promedio y varianza para todas las configuraciones

df = pd.DataFrame(grilla_sgdc.cv_results_)
df[['mean_test_score','std_test_score']].sort_values('mean_test_score', ascending=False)

,mean_test_score,std_test_score
71,0.834792,0.008043
82,0.833447,0.001481
155,0.833447,0.001481
115,0.833447,0.001481
127,0.833447,0.001481
...,...,...
32,0.276317,0.218747
113,0.270257,0.196710
131,0.231891,0.118505
4,0.217736,0.062872


## Ejercicio 3: Árboles de Decisión

En este ejercicio se entrenarán árboles de decisión para predecir la variable objetivo.

Para ello, deberán utilizar la clase DecisionTreeClassifier de scikit-learn.

Documentación:
- https://scikit-learn.org/stable/modules/tree.html
  - https://scikit-learn.org/stable/modules/tree.html#tips-on-practical-use
- https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html
- https://scikit-learn.org/stable/auto_examples/tree/plot_unveil_tree_structure.html

### Ejercicio 3.1: DecisionTreeClassifier con hiperparámetros por defecto

Entrenar y evaluar el clasificador DecisionTreeClassifier usando los valores por omisión de scikit-learn para todos los parámetros. Únicamente **fijar la semilla aleatoria** para hacer repetible el experimento.

Evaluar sobre el conjunto de **entrenamiento** y sobre el conjunto de **evaluación**, reportando:
- Accuracy
- Precision
- Recall
- F1
- matriz de confusión


In [ ]:
model = DecisionTreeClassifier(random_state=0)
predicted = bad_prediction(model)
print_metricas(predicted)

accuracy train:  1.0
precision train:  1.0
recall train:  1.0
f1_score train:  1.0
matriz de confusión train:
[[1236    0]
 [   0  247]]

accuracy test:  0.88
precision test:  0.65
recall test:  0.58
f1_score test:  0.62
matriz de confusión test:
[[290  19]
 [ 26  36]]



### Ejercicio 3.2: Ajuste de Hiperparámetros

Seleccionar valores para los hiperparámetros principales del DecisionTreeClassifier. Como mínimo, probar diferentes criterios de partición (criterion), profundidad máxima del árbol (max_depth), y cantidad mínima de samples por hoja (min_samples_leaf).

Para ello, usar grid-search y 5-fold cross-validation sobre el conjunto de entrenamiento para explorar muchas combinaciones posibles de valores.

Reportar accuracy promedio y varianza para todas las configuraciones.

Para la mejor configuración encontrada, evaluar sobre el conjunto de **entrenamiento** y sobre el conjunto de **evaluación**, reportando:
- Accuracy
- Precision
- Recall
- F1
- matriz de confusión


Documentación:
- https://scikit-learn.org/stable/modules/grid_search.html
- https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html

In [ ]:
# diccionario de parámetros a evaluar en el modelo Decision Tree Classifier

grilla_param_tree = { 'criterion'   : ['gini', 'entropy'],
                     'splitter'     : ['best','random'],
                     'max_depth'        : np.random.choice(range(2, 30), 15, np.random.seed(60)),
                     'min_samples_leaf' : np.random.choice(range(2, 100), 15, np.random.seed(60)),
                     'max_features'     : ['auto', 'log2', None]
                     }

In [ ]:
# selección del mejor modelo de Decision Tree Classifier según los parámetros evaluados

model = DecisionTreeClassifier(random_state=60)

grilla_tree = GridSearchCV(model, grilla_param_tree, scoring='accuracy')

grilla_tree.fit(X_train, y_train)

mejor_tree = grilla_tree.best_estimator_
print(mejor_tree)

# entrenamiento y evaluación del mejor modelo de SGDClassifier según los parámetros evaluados

predicted = bad_prediction(mejor_tree)
print_metricas(predicted)

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=6, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=16, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=60, splitter='best')
accuracy train:  0.89
precision train:  0.76
recall train:  0.48
f1_score train:  0.59
matriz de confusión train:
[[1198   38]
 [ 128  119]]

accuracy test:  0.89
precision test:  0.87
recall test:  0.42
f1_score test:  0.57
matriz de confusión test:
[[305   4]
 [ 36  26]]



In [ ]:
# accuracy promedio y varianza para todas las configuraciones

df = pd.DataFrame(grilla_tree.cv_results_)
df[['mean_test_score','std_test_score']].sort_values('mean_test_score', ascending=False)

,mean_test_score,std_test_score
668,0.879318,0.017366
638,0.879318,0.017366
818,0.879307,0.008248
218,0.879307,0.008248
1208,0.879307,0.008248
...,...,...
605,0.832080,0.013271
2065,0.831420,0.005726
2053,0.830744,0.005093
1345,0.829393,0.007728
